# Deploy Keras Model to the Cloud
1. Download data
2. Hot Encode Categorical Columns
3. Develop Model (use: ```RMSprop(lr=, rho=, epsilon=, decay=)```)
3. Create Storage Bucket
4. Authenticate on Google Cloud Platform
5. Deploy your model on vertex ai and show that it has returned a prediction

In [1]:
!wget https://storage.googleapis.com/nicksdemobucket/bank.csv

--2022-06-13 03:49:35--  https://storage.googleapis.com/nicksdemobucket/bank.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.18.128, 173.194.69.128, 108.177.119.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.18.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4610348 (4.4M) [application/octet-stream]
Saving to: ‘bank.csv.3’

bank.csv.3          100%[===================>]   4.40M  --.-KB/s    in 0.02s   

2022-06-13 03:49:35 (181 MB/s) - ‘bank.csv.3’ saved [4610348/4610348]



In [2]:
import numpy as np
import pandas as pd
df = pd.read_csv('bank.csv', sep=";")
df.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


In [3]:
df.describe()

,age,balance,day,duration,campaign,pdays,previous
count,45211.000000,45211.000000,45211.000000,45211.000000,45211.000000,45211.000000,45211.000000
mean,40.936210,1362.272058,15.806419,258.163080,2.763841,40.197828,0.580323
std,10.618762,3044.765829,8.322476,257.527812,3.098021,100.128746,2.303441
min,18.000000,-8019.000000,1.000000,0.000000,1.000000,-1.000000,0.000000
25%,33.000000,72.000000,8.000000,103.000000,1.000000,-1.000000,0.000000
50%,39.000000,448.000000,16.000000,180.000000,2.000000,-1.000000,0.000000
75%,48.000000,1428.000000,21.000000,319.000000,3.000000,-1.000000,0.000000
max,95.000000,102127.000000,31.000000,4918.000000,63.000000,871.000000,275.000000


Hot Encode Categorical Columns

In [4]:
df.loc[df['y'] == 'no', 'y'] = 0
df.loc[df['y'] == 'yes', 'y'] = 1
df.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,0
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,0
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,0
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,0
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,0


In [5]:
df["y"]=df["y"].astype(int)
#df.dtypes

In [6]:
# Get all categorical columns
categorical = df.select_dtypes(include='object').columns
print(categorical)

Index(['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact',
       'month', 'poutcome'],
      dtype='object')


In [7]:
categorical

Index(['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact',
       'month', 'poutcome'],
      dtype='object')

In [8]:
# Hot encode all categorical columns
for i in categorical:
  new_cols = pd.get_dummies(df[i], prefix=i)
  df = df.drop(i, axis=1)
  df = df.join(new_cols)
  
df.head()

,age,balance,day,duration,campaign,pdays,previous,y,job_admin.,job_blue-collar,...,month_jun,month_mar,month_may,month_nov,month_oct,month_sep,poutcome_failure,poutcome_other,poutcome_success,poutcome_unknown
0,58,2143,5,261,1,-1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
1,44,29,5,151,1,-1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
2,33,2,5,76,1,-1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
3,47,1506,5,92,1,-1,0,0,0,1,...,0,0,1,0,0,0,0,0,0,1
4,33,1,5,198,1,-1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1


In [9]:
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler()

train, test = train_test_split(df, test_size=0.1)

X_train = min_max_scaler.fit_transform(train.drop(['y'], axis=1))
y_train = train[['y']].values

X_test = min_max_scaler.fit_transform(test.drop(['y'], axis=1))
y_test = test[['y']].values

X_train.shape, y_train.shape, X_test.shape, y_test.shape

((40689, 51), (40689, 1), (4522, 51), (4522, 1))

In [10]:
X_train

array([[0.15584416, 0.09391172, 0.13333333, ..., 1.        , 0.        ,
        0.        ],
       [0.20779221, 0.08556825, 0.86666667, ..., 0.        , 0.        ,
        1.        ],
       [0.20779221, 0.07737911, 0.9       , ..., 0.        , 0.        ,
        1.        ],
       ...,
       [0.41558442, 0.07208614, 0.2       , ..., 0.        , 0.        ,
        1.        ],
       [0.25974026, 0.07279429, 0.2       , ..., 0.        , 0.        ,
        1.        ],
       [0.22077922, 0.07434678, 0.36666667, ..., 0.        , 0.        ,
        1.        ]])

In [11]:
y_train

array([[0],
       [1],
       [0],
       ...,
       [0],
       [0],
       [0]])

Develop Model (use: RMSprop(lr=, rho=, epsilon=, decay=))

In [12]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import tensorflow as tf

tf.__version__

'2.8.2'

In [13]:
model = Sequential()

model.add(Dense(4, input_shape=(51,), activation='relu'))
model.add(Dense(1))
model.compile(loss='mean_squared_error',
                optimizer = tf.keras.optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0),
                metrics=['accuracy'])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 4)                 208       
                                                                 
 dense_1 (Dense)             (None, 1)                 5         
                                                                 
Total params: 213
Trainable params: 213
Non-trainable params: 0
_________________________________________________________________


In [15]:
model.fit(X_train, y_train, epochs=10)

Epoch 1/10
1272/1272 [==============================] - 4s 2ms/step - loss: 0.0944 - accuracy: 0.8869
Epoch 2/10
1272/1272 [==============================] - 4s 3ms/step - loss: 0.0816 - accuracy: 0.8940
Epoch 3/10
1272/1272 [==============================] - 4s 3ms/step - loss: 0.0761 - accuracy: 0.8961
Epoch 4/10
1272/1272 [==============================] - 3s 2ms/step - loss: 0.0727 - accuracy: 0.8991
Epoch 5/10
1272/1272 [==============================] - 2s 2ms/step - loss: 0.0712 - accuracy: 0.9000
Epoch 6/10
1272/1272 [==============================] - 2s 1ms/step - loss: 0.0708 - accuracy: 0.9010
Epoch 7/10
1272/1272 [==============================] - 2s 1ms/step - loss: 0.0707 - accuracy: 0.9016
Epoch 8/10
1272/1272 [==============================] - 2s 1ms/step - loss: 0.0704 - accuracy: 0.9015
Epoch 9/10
1272/1272 [==============================] - 2s 1ms/step - loss: 0.0703 - accuracy: 0.9013
Epoch 10/10
1272/1272 [==============================] - 2s 1ms/step - loss: 0.070

Create Storage Bucket

In [16]:
PROJECT_ID = "mgsc695-353202" #@param {type:"string"}

! gcloud config set project $PROJECT_ID

Updated property [core/project].


Authenticate on Google Cloud Platform

In [17]:
from google.colab import auth as google_auth
google_auth.authenticate_user()

[in the GCP console]... Create "Google Cloud Storage Bucket" in GCP


In [18]:
# Export model and save to GCS
BUCKET_NAME = "mcgillbucket_yichen_wang" #@param {type:"string"}

model.save("gs://" + BUCKET_NAME + '/mpg/model')

INFO:tensorflow:Assets written to: gs://mcgillbucket_yichen_wang/mpg/model/assets


Deploy your model on vertex ai and show that it has returned a prediction

In [19]:
%%capture
!pip3 install google-cloud-aiplatform --upgrade --user

In [20]:
import os

# Google Cloud Notebook
if os.path.exists("/opt/deeplearning/metadata/env_version"):
    USER_FLAG = "--user"
else:
    USER_FLAG = ""

! pip3 install --upgrade google-cloud-aiplatform $USER_FLAG

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [21]:
! pip3 install -U google-cloud-storage $USER_FLAG

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [22]:
if os.getenv("IS_TESTING"):
    !pip3 install --upgrade tensorflow $USER_FLAG

In [23]:
from google.cloud import aiplatform

endpoint = aiplatform.Endpoint(
    endpoint_name="projects/788749637531/locations/us-central1/endpoints/5106404678275432448"
)

In [30]:
response = endpoint.predict([X_test[0].tolist()])

print('API response: ', response)

print('Predicted MPG: ', response.predictions[0][0])

API response:  Prediction(predictions=[[0.010820359]], deployed_model_id='3522100781626949632', explanations=None)
Predicted MPG:  0.010820359
